In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd

import sys
sys.path.append('/home/oskar/nlll/NLP_Learning/NLP_Learning')  # Adjust the path according to your environment

from module import *
from functions import *
#from ..NLP_Learning.module import MultiheadAttention

torch.manual_seed(123)
torch.set_printoptions(sci_mode=False)

/home/oskar/nlll/NLP_Learning/.venv/lib64/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
GPT_config_124M = {
    'vocab_size': 50257,
    'context_length': 1024,
    'num_head': 12, 
    'emb_dim': 768,
    'n_layers': 12, 
    'dropout': 0.1,
    'qkv_bias': False
}

In [3]:
import tiktoken

tokenizer = tiktoken.get_encoding("gpt2")

batch = []

txt1 = "Every effort moves you"
txt2 = "Every day holds a"

batch.append(torch.tensor(tokenizer.encode(txt1)))
batch.append(torch.tensor(tokenizer.encode(txt2)))
batch = torch.stack(batch, dim=0)
print(batch)

tensor([[6109, 3626, 6100,  345],
        [6109, 1110, 6622,  257]])


In [4]:
model = GPTModel(GPT_config_124M)

out = model(batch)

print("Input batch:\n", batch)
print("\nOutput shape:", out.shape)
print(out)

Input batch:
 tensor([[6109, 3626, 6100,  345],
        [6109, 1110, 6622,  257]])

Output shape: torch.Size([2, 4, 50257])
tensor([[[    -0.1333,      0.0012,      0.1082,  ...,     -0.3430,
              -0.3561,     -0.1098],
         [     0.1763,     -0.5144,      0.8026,  ...,     -0.6271,
              -0.9305,      0.3440],
         [     0.4219,     -0.2349,     -0.2557,  ...,     -0.0625,
              -0.6108,     -0.2136],
         [     0.1196,      0.5278,      0.2564,  ...,      0.2090,
               0.2731,      0.3246]],

        [[    -0.7906,     -0.3488,     -0.0550,  ...,     -0.5934,
              -0.4162,     -0.1623],
         [     0.1259,     -1.4402,      1.1246,  ...,     -0.9579,
              -0.7318,      0.1409],
         [    -0.3515,     -0.2160,     -0.3976,  ...,     -0.3987,
              -0.5907,      0.5050],
         [     0.0605,      1.1985,      0.6972,  ...,     -0.6066,
              -0.1506,     -0.5794]]], grad_fn=<UnsafeViewBackward0>)


In [5]:
start_context = 'Hello, I am'
encoded = tokenizer.encode(start_context)

encoded_tensor = torch.tensor(encoded).unsqueeze(0)

model.eval()
out = generate_text_simple(
    model = model,
    idx = encoded_tensor,
    max_new_tokens = 6,
    context_size = GPT_config_124M['context_length']
)
print('Output:', out)
print('Output lenght:', len(out[0]))

Output: tensor([[15496,    11,   314,   716, 49187, 27829, 47568,  8969, 39126, 39603]])
Output lenght: 10


In [6]:
decoded_text = tokenizer.decode(out[0].squeeze().tolist())
print(decoded_text)

Hello, I am correctional 161popular iOSenglish slamming


In [7]:
import torch
GPT_CONFIG_124M = {
        "vocab_size": 50257,
        "context_length": 256, #A
        "emb_dim": 768,
        "num_head": 12,
        "n_layers": 12,
        "dropout": 0.1,
        #B
        "qkv_bias": False
        }
torch.manual_seed(123)
model = GPTModel(GPT_CONFIG_124M)
model.eval()

GPTModel(
  (tok_emb): Embedding(50257, 768)
  (pos_emb): Embedding(256, 768)
  (dropout): Dropout(p=0.1, inplace=False)
  (trf_blocks): Sequential(
    (0): TransformerBlock(
      (att): MultiheadAttention(
        (W_keys): Linear(in_features=768, out_features=768, bias=True)
        (W_query): Linear(in_features=768, out_features=768, bias=True)
        (W_values): Linear(in_features=768, out_features=768, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
        (out_proj): Linear(in_features=768, out_features=768, bias=True)
      )
      (ff): FeedForward(
        (l1): Linear(in_features=768, out_features=3072, bias=True)
        (l2): Linear(in_features=3072, out_features=768, bias=True)
        (gelu): GELU()
      )
      (norm1): LayerNorm()
      (norm2): LayerNorm()
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (1): TransformerBlock(
      (att): MultiheadAttention(
        (W_keys): Linear(in_features=768, out_features=768, bias=True)
        (W_quer

In [111]:
torch.set_printoptions(sci_mode=True)
with torch.no_grad(): #A
    logits = model(batch)
probas = torch.softmax(logits, dim=-1)
choices = torch.argmax(probas, dim=-1, keepdim=True)
tokenizer.decode(choices.unsqueeze(0))

TypeError: argument 'tokens': 'Tensor' object cannot be converted to 'Sequence'

In [17]:
inputs = torch.tensor([[16833, 3626, 6100], [40,1107, 588]])

targets = torch.tensor([[3626, 6100, 345 ], [588, 428, 11311]])

with torch.no_grad(): #A
    logits = model(inputs)
probas = torch.softmax(logits, dim=-1)
print(probas.shape)

token_ids = torch.argmax(probas, dim=-1, keepdim=True)
print('Token IDs', token_ids)

torch.Size([2, 3, 50257])
Token IDs tensor([[[47770],
         [44177],
         [49835]],

        [[36641],
         [26415],
         [18265]]])


In [112]:
def token_to_text(token_ids, tokenizer):
    flat = token_ids.squeeze(0) # remove batch dimension
    return tokenizer.decode(token_ids.tolist())

In [113]:
print(f"Targets batch 1: {token_to_text(targets[0], tokenizer)}")
print(f"Outputs batch 1: {token_to_text(token_ids[0].flatten(), tokenizer)}")

Targets batch 1:  effort moves you
Outputs batch 1:  solicitation Carnageoglobin


In [87]:
text_idx = 0
target_probas_1 = probas[text_idx, [0, 1, 2], targets[text_idx]]
print("Text 1:", target_probas_1)

Text 1: tensor([    0.0000,     0.0000,     0.0000])


In [89]:
targets[text_idx]

tensor([3626, 6100,  345])

In [90]:
probas

tensor([[[    0.0000,     0.0000,     0.0000,  ...,     0.0000,
              0.0000,     0.0000],
         [    0.0000,     0.0000,     0.0000,  ...,     0.0000,
              0.0000,     0.0000],
         [    0.0000,     0.0000,     0.0000,  ...,     0.0001,
              0.0000,     0.0000]],

        [[    0.0000,     0.0000,     0.0000,  ...,     0.0000,
              0.0000,     0.0000],
         [    0.0000,     0.0000,     0.0000,  ...,     0.0000,
              0.0000,     0.0000],
         [    0.0000,     0.0000,     0.0000,  ...,     0.0001,
              0.0000,     0.0000]]])

In [9]:
batch_size, context_lengt, d_in = batch.shape

model = MultiheadAttention(d_in = 3, d_out=2, num_heads=2, context_lengt = context_lengt)
model(batch)

ValueError: not enough values to unpack (expected 3, got 2)

In [ ]:
class LayerNorm(nn.Module):
    def __init__(self, emb_dim, eps=1e-5):
        super().__init__()
        self.eps = 1e-5
        self.emb_dim = emb_dim
        self.scale = nn.Parameter(torch.ones(self.emb_dim))
        self.bias = nn.Parameter(torch.zeros(self.emb_dim))

    def forward(self, x):
        mean = x.mean(dim=-1, keepdim=True)
        var = x.var(dim=-1, keepdim=True, unbiased=False)
        norm_x = (x - mean) / torch.sqrt(var + self.eps)
        return self.scale * norm_x + self.bias


ln = LayerNorm(emb_dim=5)

out_ln = ln(batch_example)
print(out_ln)

print(out_ln.mean(dim=-1, keepdim=True))
print(out_ln.var(dim=-1, keepdim=True))

In [ ]:
ln = LayerNorm(emb_dim=5)

out_ln = ln(batch_example)
print(out_ln)

print(out_ln.mean(dim=-1, keepdim=True))
print(out_ln.var(dim=-1, keepdim=True))

tensor([[ 0.5528,  1.0693, -0.0223,  0.2656, -1.8654],
        [ 0.9087, -1.3767, -0.9564,  1.1304,  0.2940]], grad_fn=<AddBackward0>)
tensor([[-2.9802e-08],
        [ 0.0000e+00]], grad_fn=<MeanBackward1>)
tensor([[1.2499],
        [1.2500]], grad_fn=<VarBackward0>)


In [ ]:
class ExampleDeepNeuralNetwork(nn.Module):
    def __init__(self, layer_size, use_shortcut=False):
        super().__init__()
        self.use_shortcut = use_shortcut

        self.layers = nn.ModuleList([
            nn.Sequential(nn.Linear(layer_size[0], layer_size[1]), GELU()),
            nn.Sequential(nn.Linear(layer_size[1], layer_size[2]), GELU()),
            nn.Sequential(nn.Linear(layer_size[2], layer_size[3]), GELU()),
            nn.Sequential(nn.Linear(layer_size[3], layer_size[4]), GELU()),
            nn.Sequential(nn.Linear(layer_size[4], layer_size[5]), GELU())
        ]
        )

    def forward(self, x):
        #compute
        for layer in self.layers:
            layer_out = layer(x)

            if self.use_shortcut and x.shape == layer_out.shape:
                x = x + layer_out
            else:
                x = layer_out
        return x
        

In [ ]:
torch.set_printoptions(sci_mode=False)

In [ ]:
layer_size = [3,3,3,3,3,3,1]
model = ExampleDeepNeuralNetwork(layer_size)
sample_input = torch.tensor([[1., 0., -1.]])

def print_gradient(model, x):
    output = model(x)
    target = torch.randn_like(output)

    loss = torch.nn.MSELoss()(output, target)

    loss.backward()

    for name, param in model.named_parameters():
        if 'weight' in name:
            print(f'{name} has a param {param.grad.abs().mean().item()}')

In [ ]:
print_gradient(model, samle_input)

layers.0.0.weight has a param 0.0012179916957393289
layers.1.0.weight has a param 0.0030874121002852917
layers.2.0.weight has a param 0.0020798740442842245
layers.3.0.weight has a param 0.011124690063297749
layers.4.0.weight has a param 0.021914280951023102


In [ ]:
layer_size = [3, 3, 3, 3, 3, 1]
model = ExampleDeepNeuralNetwork(layer_size, use_shortcut=True)

print_gradient(model, samle_input)

layers.0.0.weight has a param 0.03952733799815178
layers.1.0.weight has a param 0.052864931523799896
layers.2.0.weight has a param 0.09154993295669556
layers.3.0.weight has a param 0.07208192348480225
layers.4.0.weight has a param 0.5970112085342407
